In [1]:
import math
import random
import numpy

#Team Draft Interleaving


#to be able to distinguish the document's assignment, rename the relevances by adding index number
#eg. Ranking pair:  [['HR', 'HR', 'R', 'R', 'N'], ['HR', 'R', 'R', 'N', 'N']]
# Updated pair : (['HR0', 'HR1', 'R2', 'R3', 'N4'], ['HR5', 'R6', 'R7', 'N8', 'N9'])

def preprocess(pair):
    i = 0
    updated_pair = ([], [])
    for relevance in pair[0]:
        updated_pair[0].append(relevance+ str(i))
        i+=1
    
    for relevance in pair[1]:
        updated_pair[1].append(relevance+ str(i))
        i+=1        
    
    return updated_pair


#a) create the interleaving list
def team_draft_interleave(pair):
    A = pair[0][:]
    B = pair[1][:]
    doc_list = []  #the interleaving list
    assignments = [] #holds the pair that the documents taken from, "A or B", "0 or 1" respectively
    while A or B:
        if A and B:
            first = random.randint(0,1)
        elif A:
            first = 0
        else:
            first = 1
        second = int(math.fabs(first-1))

        #pick from the first list(A) then the second list (B)
        if first == 0: #A
            doc = A[0]
            doc_list.append(doc)
            assignments.append(first)
            #delete the doc from the first and the second list
            del A[0] 
            if doc in B:
                B.remove(doc)
            
            #pick from the second list
            doc = B[0]
            doc_list.append(doc)
            assignments.append(second) 
            #delete the doc from the second list (top element) and the first list
            del B[0]
            
            if doc in A:
                A.remove(doc)
            
       #pick from the first list(B) then the second list (A)
        else: #B
            doc = B[0]
            doc_list.append(doc)
            assignments.append(first)
            del B[0]
            if doc in A:
                A.remove(doc)

            #pick from the second list
            doc = A[0]
            doc_list.append(doc)
            assignments.append(second)
            #delete the doc from the first and the second list
            del A[0] 
            if doc in B:
                B.remove(doc)
    
       
    #append the interleaved list with the assignment        
    output = []
    for i in range(len(doc_list)):
        output.append([doc_list[i],assignments[i]])
    return output



#generate a list of 'number' amount of random clicks
def random_clicks(length, number):
    clicks = random.sample(range(length), number)
    return clicks

    
#b) evaluate the interleaved list with random clicks


def team_draft_evaluation(interleaved, clicks):
    doc_list, assignments = zip(*interleaved) 
#     print(doc_list)
#     print(assignments)
    counters = [0, 0]
    for i in clicks:
        counters[assignments[i]]+=1

    return counters[0]-counters[1]


pair = [["HR", "HR", "R", "R", "N"], ["HR", "R", "R", "N", "N"]]
print ("Ranking pair: ", pair)
updated_pair = preprocess(pair)
print("Updated pair: ", updated_pair)
interleaved = team_draft_interleave(updated_pair)
print("interleaved: ", interleaved)

clicks = random_clicks(len(interleaved), 3)
print("Random Clicks: ", clicks)

result = team_draft_evaluation(interleaved, clicks)
print("Evaluation A-B (number of clicks): ", result)

Ranking pair:  [['HR', 'HR', 'R', 'R', 'N'], ['HR', 'R', 'R', 'N', 'N']]
Updated pair:  (['HR0', 'HR1', 'R2', 'R3', 'N4'], ['HR5', 'R6', 'R7', 'N8', 'N9'])
interleaved:  [['HR0', 0], ['HR5', 1], ['HR1', 0], ['R6', 1], ['R2', 0], ['R7', 1], ['R3', 0], ['N8', 1], ['N4', 0], ['N9', 1]]
Random Clicks:  [3, 8, 4]
Evaluation A-B (number of clicks):  1


In [5]:
### STEP 5 ####

# Yandex Click Log File
f = open('YandexRelPredChallenge.txt', 'r')
content = []
for line in f:
    line = line.split()
    content.append(line)
f.closed

#print(content[:10])

#each session is a query that has the list of documents retrieved and list of documents that are clicked
sessions = []  
for line in content:
    if(line[2]) == 'Q': 
        session = []
        session.append(line[5:])
        session.append([])
        sessions.append(session)
    if(line[2]) == 'C':
        c = line[-1] 
        for i in range(-1,-len(sessions) - 1, -1): #attribute the click to the last query that had this document as a result
            session = sessions[i]
            if c in session[0]:
                sessions[i][1].append(c)
                break;
print(sessions[4])





[['17562', '1627', '1626', '1623', '2091', '17559', '17563', '17558', '17561', '17560'], ['17562', '1627', '1626']]
